#with error

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [28]:
FEATURES = [
    "gd",
    "ed",
    "gp",
    #"isr",
#    "views", 
    "large", 
    "top", 
#    "speed_level",
#    "speed_level 1",
#    "speed_level 2",
#    "speed_level 3",
    "price"]

In [3]:
def get_speed_level(speed_level):
    if speed_level == 1:
        return  [1]
        #return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        return [0.4]
        #return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        return [0.1]
        #return [0,0,1]
        #print X[-1], speed_level
    else:
        return [0]
        #return [0,0,0]
        #print X[-1], speed_level

In [27]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        
        #gd
        gd = [0.060, 0.223, 0.015, 0.039, -0.003, 0.011, 8.370] #unnormed
        gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
        #gd = [1, 1, 1, 1, 1, 1, 0]
        x = [
            doc['degree'].count(u'Профессор'),
            int(doc['top']),
            len(doc['illnesses']),
            len(doc['languages']),
            int(doc['h_index']),
            int(doc['experience']),
            1
        ]
        X[-1]+= [round(sum(np.array(gd)*np.array(x)),5)] 
        #ed
        ed = [-0.552, -0.032, 0.533, 0.098, 8.945] #unnormed
        ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
        #ed = [1, 1, 1, 1, 0]
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        x = [
            len(list(set(ids_diad))),
            len(doc['procedures']['diagnostics']),
            len(list(set(ids_oper))),
            len(doc['procedures']['operations']),
            1
        ]
        X[-1]+= [round(sum(np.array(ed)*np.array(x)),5)]
        #gp
        gp = [-1.547, -0.204, 1.837, 0.139] #unnormed
        gp = [-3.094, -7.545, 11.025, 9.011]
        #gp = [1, 1, 1, 1]
        x = [
            sum([int(i['top']) for i in all_proc]), #top_proc_cli
            len(all_proc), #gen_proc_cli
            sum([int(i['top']) for i in clinic['procedures']['operations']]), #top_oper_cli
            len(clinic['procedures']['operations']), #gen_oper_cli
        ]
        X[-1]+= [round(sum(np.array(gp)*np.array(x)),5)]
        #
        c_id = int(clinic['country_id'])
        #if c_id == 1:
        #    X[-1]+=[1]
        #else:
        #    X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[get_speed_level(int(clinic['speed_level']))]
        #X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        X[-1]+=[round(int(clinic['price_level'])/3.0,4)]
        
    y=[int(clinic_estim['rating'])] * len(all_doc)
    return [X, y]

In [5]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [13]:
def normale(mat):
    for i in [0,1,2]:
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [7]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 2)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X, y = get_X_y_from(cancer_data)

In [42]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)
normale(X_1)
normale(X_2)
normale(X_3)
normale(X_4)
normale(X_5)

In [39]:
X_all

[[0.5896, 0.7552, -0.7049, 1, 1, 1, 0.6667],
 [0.3126, 0.8445, -0.7049, 1, 1, 1, 0.6667],
 [1.0, 0.4025, -0.7049, 1, 1, 1, 0.6667],
 [0.4555, 0.7668, -0.7049, 1, 1, 1, 0.6667],
 [0.5123, 0.6473, -0.7049, 1, 1, 1, 0.6667],
 [0.4313, 0.8212, -0.7049, 1, 1, 1, 0.6667],
 [0.2917, 0.1693, -0.8235, 0, 0, 0, 0.0],
 [0.2331, 0.5812, -1.0, 0, 1, 0, 0.6667],
 [0.7992, 1.0, -0.6909, 1, 1, 1, 0.6667],
 [0.7072, 0.5812, -0.6909, 1, 1, 1, 0.6667],
 [0.0681, 0.181, -0.4403, 0, 0, 0, 1.0],
 [0.2817, 0.0964, -0.5925, 0, 1, 0, 1.0],
 [0.4075, 0.247, -0.4423, 0, 0, 0, 0.3333],
 [0.7992, 1.0, -0.6909, 1, 1, 1, 0.6667],
 [0.6325, 0.7668, -0.6909, 1, 1, 1, 0.6667],
 [0.5438, 0.9223, -0.6909, 1, 1, 1, 0.6667],
 [0.5797, 0.7552, -0.6909, 1, 1, 1, 0.6667],
 [0.5137, 0.2587, -1.0, 0, 1, 0, 0.6667],
 [0.2515, 0.5812, -1.0, 0, 1, 0, 0.6667],
 [0.2775, 0.0964, -1.0, 0, 1, 0, 0.6667],
 [0.5896, 0.7552, -0.7049, 1, 1, 1, 0.6667],
 [0.3126, 0.8445, -0.7049, 1, 1, 1, 0.6667],
 [1.0, 0.4025, -0.7049, 1, 1, 1, 0.6667],


In [ ]:
estims_data[3]

In [ ]:
len(cancer_data[3]['doctors'])

In [29]:
for i in range(1,10):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

1
Total: 64, train: 57, test: 7
Residual sum of squares: 7.51
Train absolute: 1.87
Test absolute: 2.42
Absolute to mean: 37.59%
Train variance score: 0.45
Test variance score: -0.77

2
Total: 64, train: 51, test: 13
Residual sum of squares: 12.09
Train absolute: 1.71
Test absolute: 2.84
Absolute to mean: 44.02%
Train variance score: 0.54
Test variance score: -0.80

3
Total: 64, train: 44, test: 20
Residual sum of squares: 11.67
Train absolute: 1.65
Test absolute: 2.63
Absolute to mean: 44.14%
Train variance score: 0.59
Test variance score: -0.63

4
Total: 64, train: 38, test: 26
Residual sum of squares: 8.93
Train absolute: 1.71
Test absolute: 2.28
Absolute to mean: 37.47%
Train variance score: 0.53
Test variance score: -0.03

5
Total: 64, train: 32, test: 32
Residual sum of squares: 7.64
Train absolute: 1.70
Test absolute: 2.16
Absolute to mean: 34.03%
Train variance score: 0.56
Test variance score: 0.04

6
Total: 64, train: 25, test: 39
Residual sum of squares: 7.24
Train absolute: 1

In [44]:
regr = process_with(X_all, y_all, info=True, ts=0.7)

Total: 64, train: 19, test: 45
Residual sum of squares: 7.06
Train absolute: 1.38
Test absolute: 2.20
Absolute to mean: 34.20%
Train variance score: 0.57
Test variance score: 0.20


In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_4)):    
    print "Real: %f \t Predicted: %f" %(y_4[i], regr.predict(X_4[i]))
    print X_4[i]
    print
    #print y_5[i]
    #print regr.predict(X_5[i])[0]

In [38]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0] #, i[1]

Features sorted by their score:
-2.47482543999
6
-0.168000 	 gd
-0.702500 	 ed
-11.670200 	 gp
1.526000 	 large
0.055700 	 top
1.526000 	 price


In [32]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

1
2
3
4
5
6


In [51]:
indices= [0, 1, 2, 3, 4, 5]
regr.predict(X[min(indices):max(indices)+1])

array([ 7.02374699,  7.27386109,  7.09661153,  6.7716579 ,  7.93270888,
        6.86126528])

In [52]:
res_X = []
res_y = []
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            print res_X[-1], res_y[-1]
        except Exception, err:
            print err, i, indices
            pass

7.28869291892 10
6.93815424392 9
7.72370685542 8
min() arg is an empty sequence 7 []
7.04999811285 6
min() arg is an empty sequence 5 []
0.662293584496 4
2.69698082182 3
min() arg is an empty sequence 2 []
1.96209630999 1
min() arg is an empty sequence 10 []
6.97261348742 9
min() arg is an empty sequence 8 []
7.93625276622 7
min() arg is an empty sequence 6 []
7.28869291892 5
5.38576124614 4
min() arg is an empty sequence 3 []
2.02322127069 2
min() arg is an empty sequence 1 []
min() arg is an empty sequence 10 []
2.86521709995 9
6.88589493389 8
min() arg is an empty sequence 7 []
7.58489686994 6
7.10431855726 5
8.28022664763 4
min() arg is an empty sequence 3 []
1.37794019965 2
2.8307206352 1
7.32956743692 10
1.61168265816 9
8.05787303595 8
7.20099071381 7
1.26583679142 6
5.60991218217 5
2.2631307993 4
2.28584014732 3
6.86767269672 2
2.78793144009 1
7.27386108837 10
6.77165789999 9
7.9327088775 8
min() arg is an empty sequence 7 []
6.86126528263 6
min() arg is an empty sequence 5 []
m

In [54]:
for x in res_X:
    print x

7.28869291892
6.93815424392
7.72370685542
7.04999811285
0.662293584496
2.69698082182
1.96209630999
6.97261348742
7.93625276622
7.28869291892
5.38576124614
2.02322127069
2.86521709995
6.88589493389
7.58489686994
7.10431855726
8.28022664763
1.37794019965
2.8307206352
7.32956743692
1.61168265816
8.05787303595
7.20099071381
1.26583679142
5.60991218217
2.2631307993
2.28584014732
6.86767269672
2.78793144009
7.27386108837
6.77165789999
7.9327088775
6.86126528263
2.45790273796
1.91311435231


In [ ]:
plt.plot([i[22] for i in X])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)